# Stokes Sinker

Demonstration example for setting up particle swarms with different material properties. This system consists of a dense, high viscosity sphere falling through a background lower density and viscosity fluid. 

In [1]:
from underworld import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis

loaded rc file /home/jovyan/workspace/codes/UWGeodynamics_use_with_docker/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
u = GEO.UnitRegistry

In [3]:
velocity = 1.0 * u.centimeter / u.hour
model_length = 2. * u.meter
model_height = 1. * u.meter
refViscosity = 1e6 * u.pascal * u.second
bodyforce = 200 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_height
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [4]:
Model = GEO.Model(elementRes=(16, 16, 16), 
                  minCoord=(-1. * u.meter, -1. * u.meter, -50. * u.centimeter), 
                  maxCoord=(1. * u.meter, 1. * u.meter, 50. * u.centimeter),
                  gravity =(0.0, 0.0, -9.81 * u.meter / u.second**2) )

In [5]:
Model.outputDir = "1_11_StokesSinker3D"

In [6]:
lightMaterial = Model.add_material(name="Light", shape=GEO.shapes.Layer3D(top=Model.top, bottom=Model.bottom))
heavyMaterial = Model.add_material(name="Heavy", shape=GEO.shapes.Sphere(center=(0., 0., 20.*u.centimetre), radius=20. * u.centimetre))

In [7]:
lightMaterial.density = 10 * u.kilogram / u.metre**3
heavyMaterial.density = 500 * u.kilogram / u.metre**3

lightMaterial.viscosity = GEO.ConstantViscosity(1e6 * u.pascal * u.second)
heavyMaterial.viscosity = GEO.ConstantViscosity(1e6 * u.pascal * u.second)

In [8]:
Model.set_velocityBCs(left=[0, None, None],
                      right=[0, None, None],
                      top=[None, None, 0.],
                      bottom=[None, None, 0], 
                      front=[None, 0., None],
                      back=[None, 0., None])

In [ ]:
Fig = vis.Figure(resolution=(1200,600))
Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.5)
Fig.window()

In [10]:
Model.init_model()

In [ ]:
Model.run_for(3.0 * u.hours, checkpoint_interval=0.1 * u.hours)